In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import matplotlib as mpl
from matplotlib.patches import Circle, Rectangle, Arc
import seaborn as sns

from sklearn.metrics import accuracy_score, log_loss
import xgboost as xgb
from sklearn.model_selection import GroupKFold

plt.style.use("fivethirtyeight")
mypal = plt.rcParams["axes.prop_cycle"].by_key()["color"]  

In [2]:
MRegularSeasonCompactResults = pd.read_csv('../../data/MRegularSeasonCompactResults.csv')
WRegularSeasonCompactResults = pd.read_csv('../../data/WRegularSeasonCompactResults.csv')
MRegularSeasonDetailedResults = pd.read_csv('../../data/MRegularSeasonDetailedResults.csv')
WRegularSeasonDetailedResults = pd.read_csv('../../data/WRegularSeasonDetailedResults.csv')
MNCAATourneyCompactResults = pd.read_csv('../../data/MNCAATourneyCompactResults.csv')
WNCAATourneyCompactResults = pd.read_csv('../../data/WNCAATourneyCompactResults.csv')
MNCAATourneyDetailedResults = pd.read_csv('../../data/MNCAATourneyDetailedResults.csv') 
WNCAATourneyDetailedResults = pd.read_csv('../../data/WNCAATourneyDetailedResults.csv')
MNCAATourneySeeds = pd.read_csv('../../data/MNCAATourneySeeds.csv')
WNCAATourneySeeds = pd.read_csv('../../data/WNCAATourneySeeds.csv')
MGameCities = pd.read_csv('../../data/MGameCities.csv') 
WGameCities = pd.read_csv('../../data/WGameCities.csv')
MSeasons = pd.read_csv('../../data/MSeasons.csv') 
WSeasons = pd.read_csv('../../data/WSeasons.csv')

In [3]:
df_seeds = pd.concat(
    [
        MNCAATourneySeeds.assign(League="M"),
        WNCAATourneySeeds.assign(League="W"),
    ],
).reset_index(drop=True)

df_season_results = pd.concat(
    [
        MRegularSeasonDetailedResults.assign(League="M"),
        WRegularSeasonDetailedResults.assign(League="W"),
    ]
).reset_index(drop=True)



In [4]:
df_tourney_results = pd.concat(
    [
        MNCAATourneyDetailedResults.assign(League="M"),
        WNCAATourneyDetailedResults.assign(League="W"),
    ]
).reset_index(drop=True)

In [5]:
df_season_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,League
0,2003,10,1104,68,1328,62,N,0,27,58,...,16,22,10,22,8,18,9,2,20,M
1,2003,10,1272,70,1393,63,N,0,26,62,...,9,20,20,25,7,12,8,6,16,M
2,2003,11,1266,73,1437,61,N,0,24,58,...,14,23,31,22,9,12,2,5,23,M
3,2003,11,1296,56,1457,50,N,0,18,38,...,8,15,17,20,9,19,4,3,23,M
4,2003,11,1400,77,1208,71,N,0,30,61,...,17,27,21,15,12,10,7,1,14,M


In [6]:
df_season_results.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
       'League'],
      dtype='object')

In [7]:
# Calculate Field Goal Percentage (FG%)
df_season_results["WFG%"] = df_season_results["WFGM"] / df_season_results["WFGA"]
df_season_results["LFG%"] = df_season_results["LFGM"] / df_season_results["LFGA"]

# Calculate Three-Point Percentage (3P%)
df_season_results["W3P%"] = df_season_results["WFGM3"] / df_season_results["WFGA3"]
df_season_results["L3P%"] = df_season_results["LFGM3"] / df_season_results["LFGA3"]

# Calculate Free Throw Percentage (FT%)
df_season_results["WFT%"] = df_season_results["WFTM"] / df_season_results["WFTA"]
df_season_results["LFT%"] = df_season_results["LFTM"] / df_season_results["LFTA"]

# Calculate Possessions per Game
df_season_results["WPoss"] = (
    df_season_results["WFGA"] + 0.44 * df_season_results["WFTA"] - df_season_results["WOR"] + df_season_results["WTO"]
)
df_season_results["LPoss"] = (
    df_season_results["LFGA"] + 0.44 * df_season_results["LFTA"] - df_season_results["LOR"] + df_season_results["LTO"]
)

# Calculate Offensive Efficiency (Points per Possession)
df_season_results["WOffEff"] = df_season_results["WScore"] / df_season_results["WPoss"]
df_season_results["LOffEff"] = df_season_results["LScore"] / df_season_results["LPoss"]

# Calculate Defensive Efficiency (Opponent Points per Possession)
df_season_results["WDefEff"] = df_season_results["LScore"] / df_season_results["WPoss"]
df_season_results["LDefEff"] = df_season_results["WScore"] / df_season_results["LPoss"]

# Calculate Rebound Percentage (REB%)
df_season_results["WREB%"] = (df_season_results["WOR"] + df_season_results["WDR"]) / (
    df_season_results["WOR"] + df_season_results["WDR"] + df_season_results["LOR"] + df_season_results["LDR"]
)
df_season_results["LREB%"] = (df_season_results["LOR"] + df_season_results["LDR"]) / (
    df_season_results["WOR"] + df_season_results["WDR"] + df_season_results["LOR"] + df_season_results["LDR"]
)

# Offensive Rebound Percentage (OREB%)
df_season_results["WOREB%"] = df_season_results["WOR"] / (df_season_results["WOR"] + df_season_results["LDR"])
df_season_results["LOREB%"] = df_season_results["LOR"] / (df_season_results["LOR"] + df_season_results["WDR"])

# Defensive Rebound Percentage (DREB%)
df_season_results["WDREB%"] = df_season_results["WDR"] / (df_season_results["WDR"] + df_season_results["LOR"])
df_season_results["LDREB%"] = df_season_results["LDR"] / (df_season_results["LDR"] + df_season_results["WOR"])

df_season_results.fillna(0, inplace=True)

df_season_results.head()


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,WOffEff,LOffEff,WDefEff,LDefEff,WREB%,LREB%,WOREB%,LOREB%,WDREB%,LDREB%
0,2003,10,1104,68,1328,62,N,0,27,58,...,0.907635,0.877193,0.827549,0.962083,0.542857,0.457143,0.388889,0.294118,0.705882,0.611111
1,2003,10,1272,70,1393,63,N,0,26,62,...,1.023991,0.929204,0.921592,1.032448,0.488636,0.511364,0.375000,0.416667,0.583333,0.625000
2,2003,11,1266,73,1437,61,N,0,24,58,...,1.144918,0.951341,0.956713,1.138490,0.447917,0.552083,0.435897,0.543860,0.456140,0.564103
3,2003,11,1296,56,1457,50,N,0,18,38,...,0.971548,0.868056,0.867453,0.972222,0.403226,0.596774,0.230769,0.472222,0.527778,0.769231
4,2003,11,1400,77,1208,71,N,0,30,61,...,1.208412,1.129135,1.114250,1.224555,0.520000,0.480000,0.531250,0.488372,0.511628,0.468750


In [8]:
df_season_results.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
       'League', 'WFG%', 'LFG%', 'W3P%', 'L3P%', 'WFT%', 'LFT%', 'WPoss',
       'LPoss', 'WOffEff', 'LOffEff', 'WDefEff', 'LDefEff', 'WREB%', 'LREB%',
       'WOREB%', 'LOREB%', 'WDREB%', 'LDREB%'],
      dtype='object')

In [9]:
df_team_season_results = pd.concat(
    [
        df_season_results[["Season", "League", "WTeamID", "DayNum", "WScore", "LScore", 'WFG%', 'LFG%', 'W3P%', 'L3P%', 'WFT%', 'LFT%', 'WPoss','LPoss', 'WOffEff', 'LOffEff', 'WDefEff', 'LDefEff', 'WREB%', 'LREB%','WOREB%', 'LOREB%', 'WDREB%', 'LDREB%']]
        .assign(GameResult="W")
        .rename(
            columns={"WTeamID": "TeamID", "WScore": "TeamScore", "LScore": "OppScore"}
        ),
        df_season_results[["Season", "League", "LTeamID", "DayNum", "WScore", "LScore", 'WFG%', 'LFG%', 'W3P%', 'L3P%', 'WFT%', 'LFT%', 'WPoss','LPoss', 'WOffEff', 'LOffEff', 'WDefEff', 'LDefEff', 'WREB%', 'LREB%','WOREB%', 'LOREB%', 'WDREB%', 'LDREB%']]
        .assign(GameResult="L")
        .rename(
            columns={"LTeamID": "TeamID", "LScore": "TeamScore", "WScore": "OppScore"}
        ),
    ]
).reset_index(drop=True)

In [10]:
df_team_season_results.tail() 

,Season,League,TeamID,DayNum,TeamScore,OppScore,WFG%,LFG%,W3P%,L3P%,...,LOffEff,WDefEff,LDefEff,WREB%,LREB%,WOREB%,LOREB%,WDREB%,LDREB%,GameResult
396743,2025,W,3416,106,58,63,0.369565,0.365079,0.235294,0.269231,...,0.894510,0.887936,0.971622,0.462687,0.537313,0.192308,0.365854,0.634146,0.807692,L
396744,2025,W,3428,106,64,68,0.365079,0.431034,0.260870,0.217391,...,0.940623,0.915332,0.999412,0.623188,0.376812,0.500000,0.242424,0.757576,0.500000,L
396745,2025,W,3194,106,39,72,0.476190,0.181818,0.416667,0.176471,...,0.548369,0.559060,1.012373,0.602740,0.397260,0.166667,0.093023,0.906977,0.833333,L
396746,2025,W,3150,106,52,70,0.423729,0.310345,0.235294,0.294118,...,0.720621,0.733221,0.970067,0.550725,0.449275,0.250000,0.189189,0.810811,0.750000,L
396747,2025,W,3398,106,52,73,0.421053,0.333333,0.357143,0.157895,...,0.697051,0.690388,0.978552,0.521127,0.478873,0.371429,0.333333,0.666667,0.628571,L


In [11]:
# Score Differential
df_team_season_results["ScoreDiff"] = (
    df_team_season_results["TeamScore"] - df_team_season_results["OppScore"]
)
df_team_season_results["Win"] = (df_team_season_results["GameResult"] == "W").astype(
    "int"
)

In [12]:
df_team_season_results.sample(10, random_state=529)

,Season,League,TeamID,DayNum,TeamScore,OppScore,WFG%,LFG%,W3P%,L3P%,...,LDefEff,WREB%,LREB%,WOREB%,LOREB%,WDREB%,LDREB%,GameResult,ScoreDiff,Win
380131,2022,W,3262,100,65,69,0.571429,0.354839,0.500000,0.347826,...,0.939031,0.500000,0.500000,0.222222,0.343750,0.656250,0.777778,L,-4,0
138690,2014,W,3206,17,97,62,0.522388,0.233766,0.285714,0.176471,...,1.152019,0.630000,0.370000,0.628571,0.369231,0.630769,0.371429,W,35,1
123193,2011,W,3424,18,66,48,0.396226,0.372549,0.166667,0.100000,...,0.943936,0.591549,0.408451,0.441176,0.270270,0.729730,0.558824,W,18,1
374972,2021,W,3290,89,63,81,0.523077,0.372881,0.352941,0.461538,...,1.108981,0.645161,0.354839,0.428571,0.176471,0.823529,0.571429,L,-18,0
187208,2023,W,3253,101,58,57,0.421053,0.416667,0.294118,0.272727,...,0.936693,0.500000,0.500000,0.379310,0.370370,0.629630,0.620690,W,1,1
338037,2014,W,3180,44,47,50,0.308824,0.354167,0.090909,0.368421,...,0.787154,0.535714,0.464286,0.415094,0.258065,0.741935,0.584906,L,-3,0
81860,2018,M,1460,126,59,53,0.327273,0.322581,0.272727,0.240000,...,0.930599,0.530864,0.469136,0.289474,0.255814,0.744186,0.710526,W,6,1
73990,2017,M,1192,75,59,47,0.370370,0.421053,0.350000,0.333333,...,1.013049,0.546875,0.453125,0.333333,0.178571,0.821429,0.666667,W,12,1
133151,2013,W,3130,8,71,52,0.483871,0.392857,0.321429,0.250000,...,1.005096,0.430769,0.569231,0.218750,0.363636,0.636364,0.781250,W,19,1
73562,2017,M,1450,65,75,62,0.538462,0.470588,0.294118,0.526316,...,1.299376,0.550000,0.450000,0.291667,0.277778,0.722222,0.708333,W,13,1


In [13]:
# Compute differences before aggregation
df_team_season_results["FGDiff"] = df_team_season_results["WFG%"] - df_team_season_results["LFG%"]
df_team_season_results["3PDiff"] = df_team_season_results["W3P%"] - df_team_season_results["L3P%"]
df_team_season_results["FTDiff"] = df_team_season_results["WFT%"] - df_team_season_results["LFT%"]
df_team_season_results["PossDiff"] = df_team_season_results["WPoss"] - df_team_season_results["LPoss"]
df_team_season_results["OffEffDiff"] = df_team_season_results["WOffEff"] - df_team_season_results["LOffEff"]
df_team_season_results["DefEffDiff"] = df_team_season_results["WDefEff"] - df_team_season_results["LDefEff"]
df_team_season_results["REBDiff"] = df_team_season_results["WREB%"] - df_team_season_results["LREB%"]
df_team_season_results["OREBDiff"] = df_team_season_results["WOREB%"] - df_team_season_results["LOREB%"]
df_team_season_results["DREBDiff"] = df_team_season_results["WDREB%"] - df_team_season_results["LDREB%"]

# Now aggregate without lambda functions
team_season_agg = (
    df_team_season_results.groupby(["Season", "TeamID", "League"])
    .agg(
        AvgScoreDiff=("ScoreDiff", "mean"),
        MedianScoreDiff=("ScoreDiff", "median"),
        MinScoreDiff=("ScoreDiff", "min"),
        MaxScoreDiff=("ScoreDiff", "max"),
        Wins=("Win", "sum"),
        Losses=("GameResult", lambda x: (x == "L").sum()),
        WinPercentage=("Win", "mean"),
        AvgFGDiff=("FGDiff", "mean"),
        Avg3PDiff=("3PDiff", "mean"),
        AvgFTDiff=("FTDiff", "mean"),
        AvgPossDiff=("PossDiff", "mean"),
        AvgOffEffDiff=("OffEffDiff", "mean"),
        AvgDefEffDiff=("DefEffDiff", "mean"),
        AvgREBDiff=("REBDiff", "mean"),
        AvgOREBDiff=("OREBDiff", "mean"),
        AvgDREBDiff=("DREBDiff", "mean"),
    )
    .reset_index()
)


In [14]:
team_season_agg.head(10)

,Season,TeamID,League,AvgScoreDiff,MedianScoreDiff,MinScoreDiff,MaxScoreDiff,Wins,Losses,WinPercentage,AvgFGDiff,Avg3PDiff,AvgFTDiff,AvgPossDiff,AvgOffEffDiff,AvgDefEffDiff,AvgREBDiff,AvgOREBDiff,AvgDREBDiff
0,2003,1102,M,0.250000,-3.0,-32,38,12,16,0.428571,0.111861,0.128053,-0.000593,-0.521429,0.249752,-0.230860,0.082889,0.037421,0.037421
1,2003,1103,M,0.629630,-2.0,-16,33,13,14,0.481481,0.084206,0.091192,-0.022898,-0.703704,0.133690,-0.108770,0.028472,0.008092,0.008092
2,2003,1104,M,4.285714,6.0,-19,28,17,11,0.607143,0.074119,0.056312,-0.010742,-0.765714,0.189152,-0.166167,0.037846,0.027269,0.027269
3,2003,1105,M,-4.884615,-3.5,-42,34,7,19,0.269231,0.069261,0.075430,-0.011472,-1.132308,0.173335,-0.140921,0.067164,0.050722,0.050722
4,2003,1106,M,-0.142857,-1.0,-26,26,13,15,0.464286,0.088609,0.035901,-0.018761,-1.192857,0.163059,-0.128522,0.090719,0.057546,0.057546
5,2003,1107,M,-9.785714,-9.5,-35,8,7,21,0.250000,0.075527,0.031720,-0.001668,0.291429,0.174041,-0.182471,0.162297,0.145580,0.145580
6,2003,1108,M,-4.030303,-4.0,-33,29,14,19,0.424242,0.076037,0.095821,0.065832,-0.863030,0.173596,-0.149034,0.004566,-0.015571,-0.015571
7,2003,1110,M,4.466667,3.0,-15,34,16,14,0.533333,0.057972,0.082093,0.001538,-0.645333,0.185943,-0.164168,0.055354,0.046388,0.046388
8,2003,1111,M,1.538462,3.5,-37,31,16,10,0.615385,0.090036,0.119614,0.054974,-0.498462,0.186792,-0.173188,0.063453,0.026843,0.026843
9,2003,1112,M,14.964286,13.0,-7,39,25,3,0.892857,0.069136,0.064768,0.101062,-0.418571,0.211275,-0.200648,0.088086,0.074498,0.074498


In [15]:
df_seeds["ChalkSeed"] = (
    df_seeds["Seed"].str.replace("a", "").str.replace("b", "").str[1:].astype("int")
)

team_season_agg = team_season_agg.merge(
    df_seeds, on=["Season", "TeamID", "League"], how="left"
)

In [16]:
# Calculate Field Goal Percentage (FG%)
df_tourney_results["WFG%"] = df_tourney_results["WFGM"] / df_tourney_results["WFGA"]
df_tourney_results["LFG%"] = df_tourney_results["LFGM"] / df_tourney_results["LFGA"]

# Calculate Three-Point Percentage (3P%)
df_tourney_results["W3P%"] = df_tourney_results["WFGM3"] / df_tourney_results["WFGA3"]
df_tourney_results["L3P%"] = df_tourney_results["LFGM3"] / df_tourney_results["LFGA3"]

# Calculate Free Throw Percentage (FT%)
df_tourney_results["WFT%"] = df_tourney_results["WFTM"] / df_tourney_results["WFTA"]
df_tourney_results["LFT%"] = df_tourney_results["LFTM"] / df_tourney_results["LFTA"]

# Calculate Possessions per Game
df_tourney_results["WPoss"] = (
    df_tourney_results["WFGA"] + 0.44 * df_tourney_results["WFTA"] - df_tourney_results["WOR"] + df_tourney_results["WTO"]
)
df_tourney_results["LPoss"] = (
    df_tourney_results["LFGA"] + 0.44 * df_tourney_results["LFTA"] - df_tourney_results["LOR"] + df_tourney_results["LTO"]
)

# Calculate Offensive Efficiency (Points per Possession)
df_tourney_results["WOffEff"] = df_tourney_results["WScore"] / df_tourney_results["WPoss"]
df_tourney_results["LOffEff"] = df_tourney_results["LScore"] / df_tourney_results["LPoss"]

# Calculate Defensive Efficiency (Opponent Points per Possession)
df_tourney_results["WDefEff"] = df_tourney_results["LScore"] / df_tourney_results["WPoss"]
df_tourney_results["LDefEff"] = df_tourney_results["WScore"] / df_tourney_results["LPoss"]

# Calculate Rebound Percentage (REB%)
df_tourney_results["WREB%"] = (df_tourney_results["WOR"] + df_tourney_results["WDR"]) / (
    df_tourney_results["WOR"] + df_tourney_results["WDR"] + df_tourney_results["LOR"] + df_tourney_results["LDR"]
)
df_tourney_results["LREB%"] = (df_tourney_results["LOR"] + df_tourney_results["LDR"]) / (
    df_tourney_results["WOR"] + df_tourney_results["WDR"] + df_tourney_results["LOR"] + df_tourney_results["LDR"]
)

# Offensive Rebound Percentage (OREB%)
df_tourney_results["WOREB%"] = df_tourney_results["WOR"] / (df_tourney_results["WOR"] + df_tourney_results["LDR"])
df_tourney_results["LOREB%"] = df_tourney_results["LOR"] / (df_tourney_results["LOR"] + df_tourney_results["WDR"])

# Defensive Rebound Percentage (DREB%)
df_tourney_results["WDREB%"] = df_tourney_results["WDR"] / (df_tourney_results["WDR"] + df_tourney_results["LOR"])
df_tourney_results["LDREB%"] = df_tourney_results["LDR"] / (df_tourney_results["LDR"] + df_tourney_results["WOR"])

df_tourney_results.fillna(0, inplace=True)

df_tourney_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,WOffEff,LOffEff,WDefEff,LDefEff,WREB%,LREB%,WOREB%,LOREB%,WDREB%,LDREB%
0,2003,134,1421,92,1411,84,N,1,32,69,...,1.172871,1.068159,1.070882,1.169888,0.494382,0.505618,0.333333,0.361702,0.638298,0.666667
1,2003,136,1112,80,1436,51,N,0,31,66,...,1.036807,0.670347,0.660964,1.051525,0.580247,0.419753,0.297297,0.181818,0.818182,0.702703
2,2003,136,1113,84,1272,71,N,0,31,59,...,1.241135,1.010820,1.049054,1.195900,0.468354,0.531646,0.312500,0.425532,0.574468,0.687500
3,2003,136,1141,79,1166,73,N,0,29,53,...,1.112676,0.980129,1.028169,1.060687,0.500000,0.500000,0.392857,0.411765,0.588235,0.607143
4,2003,136,1143,76,1301,74,N,1,27,64,...,1.099537,1.075581,1.070602,1.104651,0.513514,0.486486,0.409091,0.333333,0.666667,0.590909


In [17]:
df_team_tourney_results = pd.concat(
    [
        df_tourney_results[["Season", "League", "WTeamID", "LTeamID", "DayNum", "WScore", "LScore", 'WFG%', 'LFG%', 'W3P%', 'L3P%', 'WFT%', 'LFT%', 'WPoss','LPoss', 'WOffEff', 'LOffEff', 'WDefEff', 'LDefEff', 'WREB%', 'LREB%','WOREB%', 'LOREB%', 'WDREB%', 'LDREB%']]
        .assign(GameResult="W")
        .rename(
            columns={"WTeamID": "TeamID", "LTeamId": "OppTeamID",  "WScore": "TeamScore", "LScore": "OppScore"}
        ),
        df_tourney_results[["Season", "League", "LTeamID", "WTeamID", "DayNum", "WScore", "LScore", 'WFG%', 'LFG%', 'W3P%', 'L3P%', 'WFT%', 'LFT%', 'WPoss','LPoss', 'WOffEff', 'LOffEff', 'WDefEff', 'LDefEff', 'WREB%', 'LREB%','WOREB%', 'LOREB%', 'WDREB%', 'LDREB%']]
        .assign(GameResult="L")
        .rename(
            columns={"LTeamID": "TeamID", "WTeamID": "OppTeamID", "LScore": "TeamScore", "WScore": "OppScore"}
        ),
    ]
).reset_index(drop=True)

df_team_tourney_results["Win"] = (df_team_tourney_results["GameResult"] == "W").astype(
    "int"
)

In [18]:
df_historic_tourney_features = df_team_tourney_results.merge(
    team_season_agg[
        [
            "Season", "League", "TeamID", "WinPercentage", "MedianScoreDiff", "ChalkSeed",
            "AvgFGDiff", "Avg3PDiff", "AvgFTDiff", "AvgPossDiff", "AvgOffEffDiff",
            "AvgDefEffDiff", "AvgREBDiff", "AvgOREBDiff", "AvgDREBDiff"
        ]
    ],
    on=["Season", "League", "TeamID"],
    how="left",
).merge(
    team_season_agg[
        [
            "Season", "League", "TeamID", "WinPercentage", "MedianScoreDiff", "ChalkSeed",
            "AvgFGDiff", "Avg3PDiff", "AvgFTDiff", "AvgPossDiff", "AvgOffEffDiff",
            "AvgDefEffDiff", "AvgREBDiff", "AvgOREBDiff", "AvgDREBDiff"
        ]
    ].rename(
        columns={
            "TeamID": "OppTeamID",
            "WinPercentage": "OppWinPercentage",
            "MedianScoreDiff": "OppMedianScoreDiff",
            "ChalkSeed": "OppChalkSeed",
            "AvgFGDiff": "OppAvgFGDiff",
            "Avg3PDiff": "OppAvg3PDiff",
            "AvgFTDiff": "OppAvgFTDiff",
            "AvgPossDiff": "OppAvgPossDiff",
            "AvgOffEffDiff": "OppAvgOffEffDiff",
            "AvgDefEffDiff": "OppAvgDefEffDiff",
            "AvgREBDiff": "OppAvgREBDiff",
            "AvgOREBDiff": "OppAvgOREBDiff",
            "AvgDREBDiff": "OppAvgDREBDiff",
        }
    ),
    on=["Season", "League", "OppTeamID"],
    how="left",
)


In [19]:
df_historic_tourney_features.head()

,Season,League,TeamID,LTeamID,DayNum,TeamScore,OppScore,WFG%,LFG%,W3P%,...,OppChalkSeed,OppAvgFGDiff,OppAvg3PDiff,OppAvgFTDiff,OppAvgPossDiff,OppAvgOffEffDiff,OppAvgDefEffDiff,OppAvgREBDiff,OppAvgOREBDiff,OppAvgDREBDiff
0,2003,M,1421,1411.0,134,92,84,0.463768,0.432836,0.379310,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003,M,1112,1436.0,136,80,51,0.469697,0.312500,0.304348,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003,M,1113,1272.0,136,84,71,0.525424,0.362319,0.428571,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003,M,1141,1166.0,136,79,73,0.547170,0.450000,0.428571,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003,M,1143,1301.0,136,76,74,0.421875,0.446429,0.350000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_historic_tourney_features.columns

Index(['Season', 'League', 'TeamID', 'LTeamID', 'DayNum', 'TeamScore',
       'OppScore', 'WFG%', 'LFG%', 'W3P%', 'L3P%', 'WFT%', 'LFT%', 'WPoss',
       'LPoss', 'WOffEff', 'LOffEff', 'WDefEff', 'LDefEff', 'WREB%', 'LREB%',
       'WOREB%', 'LOREB%', 'WDREB%', 'LDREB%', 'GameResult', 'OppTeamID',
       'Win', 'WinPercentage', 'MedianScoreDiff', 'ChalkSeed', 'AvgFGDiff',
       'Avg3PDiff', 'AvgFTDiff', 'AvgPossDiff', 'AvgOffEffDiff',
       'AvgDefEffDiff', 'AvgREBDiff', 'AvgOREBDiff', 'AvgDREBDiff',
       'OppWinPercentage', 'OppMedianScoreDiff', 'OppChalkSeed',
       'OppAvgFGDiff', 'OppAvg3PDiff', 'OppAvgFTDiff', 'OppAvgPossDiff',
       'OppAvgOffEffDiff', 'OppAvgDefEffDiff', 'OppAvgREBDiff',
       'OppAvgOREBDiff', 'OppAvgDREBDiff'],
      dtype='object')

In [21]:
# Define a list of relevant features and their counterparts
features = [
    ("WinPercentage", "OppWinPercentage", "WinPctDiff"),
    ("ChalkSeed", "OppChalkSeed", "ChalkSeedDiff"),
    ("MedianScoreDiff", "OppMedianScoreDiff", "MedianScoreDiffDiff"),
    ("AvgFGDiff", "OppAvgFGDiff", "AvgFGDiffDiff"),
    ("Avg3PDiff", "OppAvg3PDiff", "Avg3PDiffDiff"),
    ("AvgFTDiff", "OppAvgFTDiff", "AvgFTDiffDiff"),
    ("AvgPossDiff", "OppAvgPossDiff", "AvgPossDiffDiff"),
    ("AvgOffEffDiff", "OppAvgOffEffDiff", "AvgOffEffDiffDiff"),
    ("AvgDefEffDiff", "OppAvgDefEffDiff", "AvgDefEffDiffDiff"),
    ("AvgREBDiff", "OppAvgREBDiff", "AvgREBDiffDiff"),
    ("AvgOREBDiff", "OppAvgOREBDiff", "AvgOREBDiffDiff"),
    ("AvgDREBDiff", "OppAvgDREBDiff", "AvgDREBDiffDiff")
]

# Iterate through each pair of columns and calculate the differences
for feature, opp_feature, diff_feature in features:
    df_historic_tourney_features[diff_feature] = (
        df_historic_tourney_features[feature] - df_historic_tourney_features[opp_feature]
    )


In [22]:
df_historic_tourney_features.sample(20)

,Season,League,TeamID,LTeamID,DayNum,TeamScore,OppScore,WFG%,LFG%,W3P%,...,MedianScoreDiffDiff,AvgFGDiffDiff,Avg3PDiffDiff,AvgFTDiffDiff,AvgPossDiffDiff,AvgOffEffDiffDiff,AvgDefEffDiffDiff,AvgREBDiffDiff,AvgOREBDiffDiff,AvgDREBDiffDiff
1002,2018,M,1139,1116.0,137,79,62,0.491525,0.357143,0.423077,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1500,2011,W,3401,3208.0,146,79,38,0.437500,0.250000,0.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2352,2004,M,1427,NaN,136,45,71,0.500000,0.238095,0.428571,...,-9.5,-0.038921,-0.036262,0.020721,-0.357905,-0.074472,0.083501,-0.050225,-0.038917,-0.038917
3889,2013,W,3329,NaN,141,59,68,0.369231,0.400000,0.238095,...,-8.0,-0.021736,-0.047569,-0.049616,0.512782,-0.049364,0.036818,0.000285,0.002533,0.002533
2238,2024,W,3328,3195.0,138,73,70,0.446429,0.373134,0.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1123,2021,M,1228,1180.0,137,78,49,0.571429,0.306452,0.285714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,2004,M,1329,1386.0,145,64,62,0.471698,0.385965,0.235294,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1253,2023,M,1116,1228.0,136,73,63,0.380952,0.384615,0.272727,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3798,2012,W,3400,NaN,138,55,68,0.480000,0.279412,0.428571,...,-4.5,0.027489,-0.041249,-0.010219,-0.061048,-0.032412,0.032118,-0.020784,-0.028834,-0.028834
1977,2019,W,3355,3346.0,138,76,65,0.400000,0.353846,0.421053,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
fivethiryeight_scores = pd.concat(
    [
        pd.read_csv("../../data/538ratingsMen.csv").assign(
            League="M"
        ),
        pd.read_csv(
            "../../data/538ratingsWomen.csv"
        ).assign(League="W"),
    ]
).reset_index(drop=True)

In [24]:
fivethiryeight_scores.head()

,Season,TeamID,TeamName,538rating,League
0,2016,1242,Kansas,94.46,M
1,2016,1314,North Carolina,93.94,M
2,2016,1438,Virginia,92.46,M
3,2016,1277,Michigan State,91.84,M
4,2016,1328,Oklahoma,89.96,M


In [25]:
df_historic_tourney_features = df_historic_tourney_features.merge(
    fivethiryeight_scores.drop("TeamName", axis=1),
    on=["Season", "League", "TeamID"],
    how="left",
).dropna(subset=["538rating"])

df_historic_tourney_features = df_historic_tourney_features.merge(
    fivethiryeight_scores.drop("TeamName", axis=1).rename(
        columns={"TeamID": "OppTeamID"}
    ),
    on=["Season", "League", "OppTeamID"],
    how="left",
    suffixes=("", "Opp"),
)

In [26]:
df_historic_tourney_features["538rating_diff"] = (
    df_historic_tourney_features["538rating"]
    - df_historic_tourney_features["538ratingOpp"]
)

In [27]:
FEATURES = [
    "WinPercentage", "MedianScoreDiff", "ChalkSeed", "OppWinPercentage", 
    "OppMedianScoreDiff", "OppChalkSeed", "WinPctDiff", "ChalkSeedDiff",
    "AvgFGDiff", "Avg3PDiff", "AvgFTDiff", "AvgPossDiff", "AvgOffEffDiff", 
    "AvgDefEffDiff", "AvgREBDiff", "AvgOREBDiff", "AvgDREBDiff", 
    "OppAvgFGDiff", "OppAvg3PDiff", "OppAvgFTDiff", "OppAvgPossDiff", 
    "OppAvgOffEffDiff", "OppAvgDefEffDiff", "OppAvgREBDiff", "OppAvgOREBDiff", 
    "OppAvgDREBDiff", "538rating_diff"
]
TARGET = "Win"

In [28]:
X = df_historic_tourney_features[FEATURES]
y = df_historic_tourney_features[TARGET]
groups = df_historic_tourney_features["Season"]
seasons = df_historic_tourney_features["Season"].unique()

In [29]:
# Setup cross-validation
gkf = GroupKFold(n_splits=df_historic_tourney_features["Season"].nunique())
cv_results = []
models = []

In [30]:
season_idx = 0
for train_index, test_index in gkf.split(X, y, groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Prepare the model
    model = xgb.XGBRegressor(
        eval_metric="logloss",
        n_estimators=1_000,
        learning_rate=0.001,
    )
    holdout_season = seasons[season_idx]
    print(f"Holdout Season: {holdout_season}")
    # Train the model
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=100)

    # Predict on the test set
    y_pred = model.predict(X_test)
    score_ll = log_loss(y_test, y_pred)
    y_pred = y_pred > 0.5
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    cv_results.append(accuracy)
    season_idx += 1
    print(f"Season {holdout_season}: {accuracy} {score_ll}")
    models.append(model)

Holdout Season: 2016
[0]	validation_0-logloss:0.69215
[100]	validation_0-logloss:0.60148
[200]	validation_0-logloss:0.52597
[300]	validation_0-logloss:0.46223
[400]	validation_0-logloss:0.40786
[500]	validation_0-logloss:0.36108
[600]	validation_0-logloss:0.32056
[700]	validation_0-logloss:0.28526
[800]	validation_0-logloss:0.25435
[900]	validation_0-logloss:0.22718
[999]	validation_0-logloss:0.20344
Season 2016: 1.0 0.2034416401948006
Holdout Season: 2017
[0]	validation_0-logloss:0.69215
[100]	validation_0-logloss:0.60148
[200]	validation_0-logloss:0.52597
[300]	validation_0-logloss:0.46223
[400]	validation_0-logloss:0.40786
[500]	validation_0-logloss:0.36108
[600]	validation_0-logloss:0.32056
[700]	validation_0-logloss:0.28526
[800]	validation_0-logloss:0.25435
[900]	validation_0-logloss:0.22718
[999]	validation_0-logloss:0.20344
Season 2017: 1.0 0.2034416401948006
Holdout Season: 2018
[0]	validation_0-logloss:0.69215
[100]	validation_0-logloss:0.60148
[200]	validation_0-logloss:0.52

In [31]:
print("Average CV Accuracy:", np.mean(cv_results))

Average CV Accuracy: 1.0


In [34]:
TEST_SEASON = 2024  # Change to 2024 when it comes out!

seeds_2024 = pd.read_csv('../../data/2024_tourney_seeds.csv') 
seeds_2024.sample(20)

,Tournament,Seed,TeamID
32,M,Y01,1345
101,W,Y06,3257
52,M,Z05,1458
82,W,X03,3301
111,W,Y16,3221
89,W,X10,3268
100,W,Y05,3160
11,M,W12,1412
90,W,X11,3453
13,M,W14,1287


In [35]:


seeds_2024["ChalkSeed"] = (
    seeds_2024["Seed"].str.replace("W", "").str.replace("X", "").str.replace("Z", "").str.replace("Y", "").str[1:].astype("int")
)

In [32]:
tourney_pairs = (
    seeds_2024.merge(seeds_2024, on=["Tournament"], suffixes=("", "Opp"))
    .assign(Season=TEST_SEASON)
    .query("TeamID != TeamIDOpp")
    .rename(columns={"Tournament": "League"})
)

# Merge with team_season_agg to add columns related to the team stats
tourney_pairs = (
    tourney_pairs.merge(
        team_season_agg[
            ["Season", "League", "TeamID", "AvgScoreDiff", "MedianScoreDiff", "MinScoreDiff", "MaxScoreDiff", 
             "Wins", "Losses", "WinPercentage", "AvgFGDiff", "Avg3PDiff", "AvgFTDiff", "AvgPossDiff", "AvgOffEffDiff",
             "AvgDefEffDiff", "AvgREBDiff", "AvgOREBDiff", "AvgDREBDiff", "Seed", "ChalkSeed"]
        ],
        on=["Season", "League", "TeamID"],
        how="left",
    )
    .merge(
        team_season_agg[
            ["Season", "League", "TeamID", "AvgScoreDiff", "MedianScoreDiff", "MinScoreDiff", "MaxScoreDiff", 
             "Wins", "Losses", "WinPercentage", "AvgFGDiff", "Avg3PDiff", "AvgFTDiff", "AvgPossDiff", "AvgOffEffDiff",
             "AvgDefEffDiff", "AvgREBDiff", "AvgOREBDiff", "AvgDREBDiff", "Seed", "ChalkSeed"]
        ].rename(
            columns={
                "TeamID": "TeamIDOpp",
                "AvgScoreDiff": "OppAvgScoreDiff", "MedianScoreDiff": "OppMedianScoreDiff", "MinScoreDiff": "OppMinScoreDiff", "MaxScoreDiff": "OppMaxScoreDiff",
                "Wins": "OppWins", "Losses": "OppLosses", "WinPercentage": "OppWinPercentage", "AvgFGDiff": "OppAvgFGDiff", "Avg3PDiff": "OppAvg3PDiff",
                "AvgFTDiff": "OppAvgFTDiff", "AvgPossDiff": "OppAvgPossDiff", "AvgOffEffDiff": "OppAvgOffEffDiff", "AvgDefEffDiff": "OppAvgDefEffDiff",
                "AvgREBDiff": "OppAvgREBDiff", "AvgOREBDiff": "OppAvgOREBDiff", "AvgDREBDiff": "OppAvgDREBDiff", "Seed": "OppSeed", "ChalkSeed": "OppChalkSeed"
            }
        ),
        on=["Season", "League", "TeamIDOpp"],
    )
    .reset_index(drop=True)
)

# Creating the OppChalkSeed column based on SeedOpp
tourney_pairs["OppChalkSeed"] = (
    tourney_pairs["SeedOpp"]
    .str.replace("a", "")
    .str.replace("b", "")
    .str[1:]
    .astype("int")
)


NameError: name 'seeds_2024' is not defined

In [ ]:
# Merge fivethiryeight_scores into tourney_pairs for both the team and opponent
tourney_pairs = tourney_pairs.merge(
    fivethiryeight_scores.drop("TeamName", axis=1),
    on=["Season", "League", "TeamID"],
    how="left",
)

tourney_pairs = tourney_pairs.merge(
    fivethiryeight_scores.drop("TeamName", axis=1).rename(
        columns={"TeamID": "TeamIDOpp"}
    ),
    on=["Season", "League", "TeamIDOpp"],
    how="left",
    suffixes=("", "Opp"),
)

# Create the diff columns
tourney_pairs["538rating_diff"] = (
    tourney_pairs["538rating"] - tourney_pairs["538ratingOpp"]
)

# Baseline Prediction based on ChalkSeed
tourney_pairs["BaselinePred"] = (
    tourney_pairs["ChalkSeed_x"] < tourney_pairs["ChalkSeed_y"]
)
# If ChalkSeed is equal, use WinPercentage to predict
tourney_pairs.loc[
    tourney_pairs["ChalkSeed_x"] == tourney_pairs["ChalkSeed_y"],
    "BaselinePred",
] = (
    tourney_pairs["WinPercentage"] > tourney_pairs["OppWinPercentage"]
)

# Create Win Percentage Diff
tourney_pairs["WinPctDiff"] = (
    tourney_pairs["WinPercentage"] - tourney_pairs["OppWinPercentage"]
)

# Chalk Seed Diff
tourney_pairs["ChalkSeedDiff"] = (
    tourney_pairs["ChalkSeed_x"] - tourney_pairs["ChalkSeed_y"]
)

# Median Score Diff Diff
tourney_pairs["MedianScoreDiffDiff"] = (
    tourney_pairs["MedianScoreDiff"] - tourney_pairs["OppMedianScoreDiff"]
)


In [ ]:
tourney_pairs["Pred"] = tourney_pairs[
    [f for f in tourney_pairs.columns if "model" in f]
].mean(axis=1)

tourney_pairs["ID"] = (
    tourney_pairs["Season"].astype("str")
    + "_"
    + tourney_pairs["TeamID"].astype("str")
    + "_"
    + tourney_pairs["TeamIDOpp"].astype("str")
)


In [ ]:
tourney_pairs.sample(30)

,League,Seed_x,TeamID,ChalkSeed_x,SeedOpp,TeamIDOpp,ChalkSeedOpp,Season,AvgScoreDiff,MedianScoreDiff,...,OppChalkSeed,538rating,538ratingOpp,538rating_diff,BaselinePred,WinPctDiff,ChalkSeedDiff,MedianScoreDiffDiff,Pred,ID
7119,W,Z02,3326,2,W01,3376,1,2023,12.500000,12.0,...,1,91.03,110.03,-19.00,True,-0.218750,-1.0,-16.0,NaN,2023_3326_3376
4431,W,W07,3279,7,X07,3235,7,2023,12.483871,14.0,...,7,87.34,91.57,-4.23,True,0.032258,-1.0,4.0,NaN,2023_3279_3235
7985,W,Z15,3263,15,Y16,3221,16,2023,-0.533333,1.5,...,16,NaN,67.66,NaN,False,-0.208602,NaN,-5.5,NaN,2023_3263_3221
3969,M,Z16,1255,16,W01,1163,1,2023,4.600000,5.0,...,1,NaN,89.24,NaN,False,-0.157576,NaN,-7.0,NaN,2023_1255_1163
3946,M,Z15,1443,15,Y09,1395,9,2023,-1.903226,-3.0,...,9,NaN,85.41,NaN,False,-0.152493,NaN,-7.0,NaN,2023_1443_1395
5166,W,X03,3301,3,W01,3376,1,2023,8.548387,9.0,...,1,89.40,110.03,-20.63,True,-0.354839,-4.0,-19.0,NaN,2023_3301_3376
2973,M,Y16,1212,16,W13,1463,13,2023,4.451613,6.0,...,13,NaN,NaN,NaN,False,0.005974,NaN,-3.0,NaN,2023_1212_1463
3631,M,Z10,1160,10,Y09,1395,9,2023,3.272727,3.0,...,9,NaN,85.41,NaN,False,-0.121212,NaN,-1.0,NaN,2023_1160_1395
5515,W,X08,3104,8,Y04,3243,4,2023,10.100000,7.0,...,4,82.91,NaN,NaN,True,0.151515,-2.0,6.0,NaN,2023_3104_3243
5594,W,X09,3199,9,Z04,3439,4,2023,13.125000,8.0,...,4,86.22,95.72,-9.50,False,-0.152218,2.0,-5.0,NaN,2023_3199_3439
